In [78]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt 

In [79]:
from mtflearn.io import load_image
from stemplot import imshow, colors_from_lbs
from mtflearn.features import ZPs, get_characteristic_length, local_max, KeyPoints, pca
from mtflearn.utils import normalize_image, remove_bg
from mtflearn.manifold import ForceGraph8
from mtflearn.denoise import denoise_svd

import re
from skimage.transform import resize
from stemplot.utils import plot_pca
from stemplot.interactive import interactive_clusters

In [80]:
import glob

size_dict = {
    'Ti':{'1028_ADF1_2':9,'1028_ADF1_11':8,'1028_ADF1_25':9,'0106_ADF1_1':17,'0106_ADF1_6':17,'0106_ADF1_8':18,'0106_ADF1_10':18,'0106_ADF1_11':21,'0106_ADF1_12':21},
    'V':{'E1_ADF1_3':14,'E1_ADF1_8':18,'E2_ADF1_3':19,'E2_ADF1_4':18,'E2_ADF1_5':14,'E2_ADF1_6':17,'E2_ADF1_7':17,'E2_ADF1_8':18,'E2_ADF1_14':26},
    'Mn':{'ADF1_2':22,'ADF1_4':18,'ADF1_7':18,'ADF1_12':22,'ADF1_18':21},
    'Co':{'ADF1_5':18,'ADF1_8':15,'ADF1_9':15,'ADF1_18':14,'ADF1_27':17,'ADF1_10':18,'ADF1_11':18}
}

def find_imgs(element):
    '''
    Given an element, this function is to find all images in folder
    :param element: 
    :return: 
    '''
    dm4_files = glob.glob(f"{element}/*.dm4")
    adf1_files = []
    adf2_files = []
    for file in dm4_files:
        if "ADF1" in file and "ADF2" not in file:
            adf1_files.append(file)
        elif "ADF2" in file and "ADF1" not in file:
            adf2_files.append(file)
    return adf1_files, adf2_files

def find_pts_and_lbs(element):
    folder_path = f"{element} pts and lbs"
    npy_files = glob.glob(f"{folder_path}/*.npy")
    pts_files = []
    lbs_files = []
    for file in npy_files:
        file_name = file.split("\\")[-1]
        if "pts" in file_name and "lbs" not in file_name:
            pts_files.append(file)
        elif "lbs" in file_name and "pts" not in file_name:
            lbs_files.append(file)
    return pts_files, lbs_files

def find_matching_images(lbs_filenames,pts_filenames, adf1_filenames,adf2_filenames):
    matching_images = []

    for idx,lbs_file in enumerate(lbs_filenames):
        files = []
        files.append(lbs_file)
        files.append(pts_filenames[idx])
        file_name = lbs_file.split('\\')[-1]
        if re.match(r'^[0-9E]', file_name):
            base_name = file_name.split('_')[0]
            if base_name[0] == 'E':
                base_name = base_name.split(' ')[0]
        else:
            base_name = ''
        match = re.search(r'(Ti|V|Mn|Co)(\d+)', file_name)
        if match:
            element = match.group(1)
            number = match.group(2)
            number = number+'.dm4'
            for idx2,img_file in enumerate(adf1_filenames):
                if base_name and (base_name in img_file and number in img_file):
                    files.append(img_file)
                    files.append(adf2_filenames[idx2])
                    file_name = (img_file.split('\\')[1]).split('.')[0]
                    files.append(size_dict[element][file_name])
                    break
                elif not base_name and number in img_file:
                    files.append(img_file)
                    files.append(adf2_filenames[idx2])
                    file_name = (img_file.split('\\')[1]).split('.')[0]
                    files.append(size_dict[element][file_name])
                    break
        matching_images.append(files)
    return matching_images

In [81]:
element = 'Ti'
img1_filenames, img2_filenames = find_imgs(element)
pts_filenames, lbs_filenames = find_pts_and_lbs(element)
matched_filenames = find_matching_images(lbs_filenames,pts_filenames, img1_filenames,img2_filenames)

In [82]:
file_idx = 5
lbs_filename, pts_filename, img1_filename, img2_filename, size = matched_filenames[file_idx]
img1 = load_image(img1_filename)
img2 = load_image(img2_filename)
file_name = (img1_filename.split('\\')[1]).split('.')[0]
if img1.shape[0] == 2048:
    img1 = resize(img1, (1024, 1024), order=0)
    img2 = resize(img2, (1024, 1024), order=0)

# normalize 
img1 = normalize_image(img1)
img2 = normalize_image(img2)

# pts
pts = np.load(pts_filename)
lbs = np.load(lbs_filename)

mask1 = np.logical_or(lbs==0, lbs==1)
mask2 = np.logical_or(lbs==2, lbs==3)

size = int(size * np.sqrt(3))
#size = int(size) - 7

In [83]:
kp = KeyPoints(pts, img2, size)
ps = kp.extract_patches(size)

In [84]:
zps = ZPs(n_max=12, size=ps.shape[1])
moments = zps.fit_transform(ps)
X = moments.select([0]).data[:, 1:]
X1 = X[mask1]
X2 = X[mask2]
lbs1 = lbs[mask1].astype(int)
lbs2 = lbs[mask2].astype(int)

pts1 = pts[mask1]
pts2 = pts[mask2]

In [85]:
plot_pca(X1, 2, lbs=lbs1)

In [86]:
plot_pca(X2, 2, lbs=lbs2)

In [87]:
from skimage.filters import gaussian

def process_Duane_feature(img, pts, X):
    aa = []
    for i in range(X.shape[1]):
        a = np.zeros_like(img)
        for (x, y), row in zip(pts, X):
            a[y, x] = row[i]
        aa.append(a)
    return np.array(aa)

def process_Duane_feature1(img, pts, X, size):
    aa = []
    for i in range(X.shape[1]):
        a = np.zeros_like(img)
        for (x, y), row in zip(pts, X):
            a[y, x] = row[i]
        a = gaussian(a, sigma=2)

        kp = KeyPoints(pts, a, size)
        ps = kp.extract_patches(size)
        zps = ZPs(n_max=12, size=ps.shape[1])
        moments = zps.fit_transform(ps)
        X = moments.select([0]).data[:, 1:]
        aa.append(X)
    return np.hstack(aa)

In [ ]:
# aa = process_Duane_feature(img2, pts1, X1)
# imshow(aa)

In [115]:
aa = process_Duane_feature1(img2, pts1, X1, size)

In [121]:
plt.imshow(img1)

In [116]:
plot_pca(aa, 2, lbs=lbs1)

In [123]:
fg = ForceGraph8(X = X1,
                 n_neighbors=10,
                 init_mode = 'pca',
                 num_iterations = 100,
                 num_negative_samples=5,
                 force_params1=(0, 2, 1, 1),
                 force_params2=(2, 5, 5, 1),
                 divide=0.1)

from time import time
t0 = time()
xy = fg.fit_transform(X1)

Construct graph from data...
Initialize 2-d embedding using PCA layout...


In [118]:
fg = ForceGraph8(X = aa,
                 n_neighbors=10,
                 init_mode = 'pca',
                 num_iterations = 100,
                 num_negative_samples=5,
                 force_params1=(0, 2, 1, 1),
                 force_params2=(2, 5, 5, 1),
                 divide=0.1)

from time import time
t0 = time()
xy = fg.fit_transform(aa)

Construct graph from data...
Initialize 2-d embedding using PCA layout...


In [124]:
app1 = interactive_clusters(xy, img2, pts[mask1], ps[mask1], lbs=lbs1, s=1, max_samples=10000)